In [66]:
from transformers import (
    AutoConfig, BertConfig, BertTokenizer, BertForRetrieval,
    BertForSequenceClassification)
import torch
import numpy as np
import pickle

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [67]:
import pandas as pd

In [68]:
df = pd.read_csv('/home/transformers-public/data_dir/doordash_test.csv')

In [69]:
df

,Title,Description,Manual Cuisine,Manual Dish Type,cuisine_label,dish_label
0,BBQ Pork Bao,Char-siu roasted pork belly,Chinese,Other,chinese,other
1,Seafood Gyoza,"Spinach skin, crab, scallop, shrimp, chilli, b...",Japanese,Other,japanese,other
2,Impossible MeltBurger,Our signature meltburger made with a plant-bas...,American,Burger,american,burger
3,Steak & Cheese,"Grilled steak, provolone, melted cheddar, cara...",American,Sandwich,american,sandwich
4,Mission,"Romaine hearts, cabbage, green chili chicken, ...",Mexican,Salad,mexican,salad
...,...,...,...,...,...,...
67,Southwest BBQ Chicken,"Chargrilled chicken breast atop garden greens,...",other,salad,other,salad
68,Santa Barbara Cobb,"Crisp shredded iceberg and Romaine lettuce, di...",other,salad,other,salad
69,1191 Maguro Sashimi,tuna,japanese,sushi,japanese,sushi
70,Salsiccia,"House-made sicilian sausage, pickled jalapeños...",NaN,NaN,italian,pizza


In [70]:
titles = df.Title.values
cuisine_label = df.cuisine_label.values
dish_label = df.dish_label.values

In [71]:
model_dir_pattern = "/home/transformers-public/output_dirs/{run_name}/checkpoint-{ckpt_num}"
# test_long_new_dataloader/checkpoint-3000"
model_dir_pattern.format(run_name="test_new_cleaned_loader",
                        ckpt_num=70000)
config = AutoConfig.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=True)
model = BertForSequenceClassification.from_pretrained(model_dir, from_tf=False, config=config,)

In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
device = "cpu"

cuda


In [73]:
model = model.to(device)

In [74]:
max_seq_length=64
test_input = tokenizer.batch_encode_plus(
    titles,
    max_length=max_seq_length,
    pad_to_max_length=True,
)

In [75]:
print(test_input)

{'input_ids': [[101, 170, 10457, 11703, 10183, 10174, 18323, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 14931, 21330, 12680, 175, 15594, 10637, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 38199, 10911, 11533, 59692, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 28780, 43213, 111, 10262, 13565, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 17032, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [76]:
model_inp = dict(test_input)
model_inp['labels'] = [0] * len(test_input['input_ids'])  # dummy labels
model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
model_out = model(**model_inp)

In [77]:
len(model_out)

3

In [78]:
print([x.shape for x in model_out])  # loss, logits, pooled_output (i.e. encoding)

[torch.Size([]), torch.Size([72, 8]), torch.Size([72, 768])]


In [79]:
predicted_logits = model_out[1].detach().cpu().numpy()
predicted_class = np.argmax(predicted_logits, axis=1)

In [80]:
print(predicted_logits)

[[ 4.77153492e+00 -1.24175501e+00 -3.16214800e+00 -7.54623175e-01
  -2.06936502e+00  2.42816269e-01  3.42153859e+00 -1.93421745e+00]
 [ 5.47104692e+00 -1.20492125e+00 -3.32169533e+00 -6.78423345e-01
  -2.16895366e+00  5.49079359e-01  2.59895849e+00 -2.07237148e+00]
 [ 6.56975126e+00 -6.17804527e-01 -2.18826199e+00 -1.24760747e+00
  -2.54976296e+00  1.02594709e+00 -5.38603723e-01 -1.19462752e+00]
 [ 6.61101675e+00 -7.67596662e-01 -2.42109942e+00 -1.24737537e+00
  -2.60189724e+00  1.06088948e+00 -1.92472190e-01 -1.31755900e+00]
 [ 6.56305408e+00 -9.74498391e-01 -2.69118476e+00 -1.12719035e+00
  -2.65755343e+00  1.07864249e+00  4.04029429e-01 -1.57024765e+00]
 [ 6.60172272e+00 -8.60764861e-01 -2.55275869e+00 -1.27240336e+00
  -2.64461470e+00  1.08395886e+00  1.00032344e-01 -1.38715637e+00]
 [ 6.61400318e+00 -8.26985598e-01 -2.48244810e+00 -1.26465893e+00
  -2.59326124e+00  1.02716434e+00 -4.22541648e-02 -1.32804275e+00]
 [ 5.64335060e+00 -1.18390775e+00 -3.13003016e+00 -6.04224801e-01
  -

In [81]:
print(predicted_class)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 6 6 6 0 0 0 0 0 0 0 0 0 6 6 6 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 6 0 0]


In [82]:
cuisines = ['american', 'chinese', 'indian', 'italian', 'japanese', 'mexican', 'other', 'thai']
print(len(cuisines))

8


In [83]:
# cuisines = pickle.load(open('../data_dir/cuisines_ordered_list.pkl', 'rb'))

In [84]:
wrong = 0
for i in range(len(titles)):
    pred_label = cuisines[predicted_class[i]]
    true_label = cuisine_label[i]
    title = titles[i]
    if pred_label != true_label:
        wrong += 1
        print(title, 'true:', true_label, 'pred:', pred_label)
print(wrong/len(titles))

BBQ Pork Bao true: chinese pred: american
Seafood Gyoza true: japanese pred: american
Mission true: mexican pred: american
Pacifica Grain Bowl true: other pred: american
Tea Leaf Salad-Laphet true: other pred: american
Spicy Noodles true: thai pred: american
Burmese Tomato Chutney true: other pred: american
Tikka Masala Burrito true: indian pred: american
Naughty Naan true: indian pred: american
Samosas true: indian pred: american
Super Meat Nachos true: mexican pred: american
Pad-King Rice Plate (Lunch) true: thai pred: other
Tom Yum Goong (Spicy Shrimp Lemongrass Soup) true: thai pred: other
Green Papaya Salad (Som Tum Thai) true: thai pred: other
Pad Woon Sen true: thai pred: other
Thai Iced Coffee true: thai pred: american
Malabar Parota true: indian pred: american
The Meats true: italian pred: american
Paneer Bhurjidosa true: indian pred: american
Tagliatelle true: italian pred: american
Vegetable Briyani true: indian pred: american
Meat Ravioli true: italian pred: american
Achar 

In [54]:
[('predicted', 'title', 'actual')] + [
    (cuisines[p], title, true)  for p, title, a in zip(predicted_class, titles, cuisine_label)]

NameError: name 'true' is not defined

## Argmax only over desired labels

In [107]:
cuisine_label_int = [cuisines.index(c) for c in cuisine_label]

In [108]:
target_label_set = list(sorted(set(cuisine_label_int)))
print(target_label_set)

[1, 7, 15, 17, 18, 22, 24, 28]


In [109]:
predicted_logits.shape

(72, 30)

In [112]:
pred_logits_restricted = predicted_logits[:, target_label_set]
pred_logits_restricted.shape
predicted_class_restricted = np.argmax(pred_logits_restricted, axis=1)

In [113]:
wrong = 0
for i in range(len(titles)):
    pcr = predicted_class_restricted[i]
    pred_label = cuisines[target_label_set[pcr]]
    true_label = cuisine_label[i]
    title = titles[i]
    if pred_label != true_label:
        wrong += 1
        print(title, 'true:', true_label, 'pred:', pred_label)
print(wrong/len(titles))

BBQ Pork Bao true: chinese pred: japanese
Steak & Cheese true: american pred: italian
Mission true: mexican pred: japanese
Spicy Noodles true: thai pred: japanese
Tikka Masala Burrito true: indian pred: mexican
Samosas true: indian pred: japanese
Jumbo Wings (10-pc Regular Pack) true: american pred: italian
Tom Yum Goong (Spicy Shrimp Lemongrass Soup) true: thai pred: japanese
Pad Woon Sen true: thai pred: japanese
Malabar Parota true: indian pred: other
The Meats true: italian pred: other
Vegetable Briyani true: indian pred: japanese
Turkey Breast Footlong Regular Sub true: other pred: american
Buffalo Chicken Sandwich true: american pred: other
Tingly Mapo Tofu true: chinese pred: japanese
Wonton Soup true: chinese pred: japanese
Nashville Looks So Good on You true: american pred: other
Kimchee Stew true: other pred: japanese
Vegan Caesar true: other pred: mexican
McCovey true: american pred: other
Gus's Special true: american pred: mexican
Daily Lentil Soup true: other pred: america

In [116]:
wrong

26

In [117]:
len(titles)

72